In [70]:
# Standard libraries
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

# Data manipulation libraries
import numpy as np
import pandas as pd

# Jupyter magic commands
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

# Custom modules
from ssa import *
from gp_regressor import GPRegressor

# Scikit-learn components
from sklearn.gaussian_process.kernels import DotProduct, RBF, WhiteKernel
from sklearn.linear_model import Ridge
from sklearn.linear_model import QuantileRegressor
from scipy import stats

# Darts - Time Series components
from darts import TimeSeries, concatenate
from darts.datasets import AirPassengersDataset
from darts.metrics import mape, rmse
from darts.utils.missing_values import extract_subseries
from darts.utils.timeseries_generation import datetime_attribute_timeseries as dt_attr

# Darts - Models
from darts.models import (
    XGBModel,
    ExponentialSmoothing,
    LinearRegressionModel,
    RandomForest,
    RegressionModel,
    TCNModel,
)

# Darts - Data processing
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import MissingValuesFiller, Scaler
import json
from afrr_preprocessing import preprocess_afrr_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
(
    afrr_pr_ts_scl_train, 
    afrr_pr_ts_scl_test, 
    afrr_pr_ts_orig_train, 
    afrr_pr_ts_orig_test, 
    exog_ts_scl_train, 
    exog_ts_scl_test,
    afrr_pr_scaler
) = preprocess_afrr_data(data_path="../data/afrr_price.parquet")


# Load the JSON file with the best parameters
with open('results/lr_hp_results.json', 'r') as f:
    lr_hyper_opt_params_dict = json.load(f)
    
with open('results/gp_hp_results.json', 'r') as f:
    gp_hyper_opt_params_dict = json.load(f)

# Extract the parameters from the loaded dictionary
lr_opt_params = lr_hyper_opt_params_dict["parameters"]
gp_opt_params = gp_hyper_opt_params_dict["parameters"]


output_chunk_length = 24
forecast_horizon = 24 
stride = 24           
start_idx = forecast_horizon 
quantiles = [0.1, 0.5, 0.9] 


### Models Definition

# ---------------- Gaussian Process Model ----------------

kernel = DotProduct() + WhiteKernel()
    
gp_model = GPRegressor(
    lags=gp_opt_params["lags"],
    lags_past_covariates=gp_opt_params["lags_past_covariates"],
    output_chunk_length=output_chunk_length,
    kernel=kernel)

gp_model.fit(afrr_pr_ts_scl_train, past_covariates=exog_ts_scl_train)

gp_backtest_forecasts = gp_model.historical_forecasts(
    series=concatenate([afrr_pr_ts_scl_train, afrr_pr_ts_scl_test], axis=0),
    past_covariates= concatenate([exog_ts_scl_train,exog_ts_scl_test], axis=0),
    start=1,  
    forecast_horizon=forecast_horizon,
    enable_optimization=True,
    num_samples=1,
    predict_likelihood_parameters=True,    
    stride=stride,
    retrain=False,
    last_points_only=False,
    verbose=True)


gp_backtest_forecasts = concatenate(gp_backtest_forecasts).with_columns_renamed(
    ['aFRR_UpCapPriceEUR_cl_mu', 'aFRR_UpCapPriceEUR_cl_sigma'],
    col_names_new=['gp_afrr_up_cap_price_mu', 'gp_afrr_up_cap_price_sigma']
).pd_dataframe()


gp_df_hat = pd.DataFrame(index=gp_backtest_forecasts.index)

for q in quantiles:
    gp_df_hat[f'gp_afrr_up_cap_price_{q}'] = afrr_pr_scaler.inverse_transform(
        TimeSeries.from_series(
            gp_backtest_forecasts['gp_afrr_up_cap_price_mu'] + 
            stats.norm.ppf(q) * gp_backtest_forecasts['gp_afrr_up_cap_price_sigma']
        )
    ).pd_series()



# ---------------- Linear Model with Quantiles ----------------

lr_quantile_models = {}
lr_backtest_forecasts = {}

for q in quantiles:
    # Initialize QuantileRegressor for this quantile
    quantile_regressor = QuantileRegressor(
        alpha=0,
        quantile=q,
        solver='highs'
    )
    
    # Create a specific model for this quantile
    lr_quantile_models[q] = RegressionModel(
        output_chunk_length=output_chunk_length,
        lags=list(range(-1, -lr_opt_params["lags_max"], -1)),
        lags_past_covariates=list(range(-1, -lr_opt_params["lags_past_covariates_max"], -1)), 
        model=quantile_regressor
    )
    
    lr_quantile_models[q].fit(afrr_pr_ts_scl_train, past_covariates=exog_ts_scl_train)
    
    lr_backtest_forecasts[q] = lr_quantile_models[q].historical_forecasts(
                                                                series=concatenate([afrr_pr_ts_scl_train, afrr_pr_ts_scl_test], axis=0),
                                                                past_covariates=concatenate([exog_ts_scl_train, exog_ts_scl_test], axis=0),
                                                                start=1,  
                                                                forecast_horizon=forecast_horizon,
                                                                enable_optimization=True,
                                                                stride=stride,
                                                                retrain=False,
                                                                last_points_only=False,
                                                                verbose=True
                                                            )
    
dfs = [
    afrr_pr_scaler.inverse_transform(
        concatenate(lr_backtest_forecasts[q]).with_columns_renamed(
            ['aFRR_UpCapPriceEUR_cl'], 
            col_names_new=[f'lr_afrr_up_cap_price_{q}']
        )
    ).pd_dataframe()
    for q in [0.1, 0.5, 0.9]
]

lr_df_hat = pd.concat(dfs, axis=1)

KeyboardInterrupt: 

In [2]:
from gp_forecaster import *

In [3]:
gp_model, gp_results = run_gp_pipeline()

----------------------------------------
EMBEDDING SUMMARY:
Embedding dimension	:  120
Trajectory dimensions	: (120, 3385)
Complete dimension	: (120, 3385)
Missing dimension     	: (120, 0)
----------------------------------------
DECOMPOSITION SUMMARY:
Rank of trajectory		: 120
Dimension of projection space	: 113
Characteristic of projection	: 0.9997


historical forecasts:   0%|          | 0/1 [00:00<?, ?it/s]